In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Dropout,
                                     UpSampling2D, BatchNormalization)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from skimage import img_as_ubyte
from datetime import datetime
from tensorflow.keras.callbacks import Callback

In [ ]:
X_train_path = "C:\work\study\image_enhancement_paper\X_train.npy"
Y_train_path = "C:\work\study\image_enhancement_paper\Y_train.npy"

# Define batch size
batch_size = 8
# Data Generator Function
def data_generator(X_path, Y_path):
    X_data = np.load(X_path, mmap_mode='r')[:5000]  # Memory-map the dataset
    Y_data = np.load(Y_path, mmap_mode='r')[:5000]

    def gen():
        for i in range(len(X_data)):
            x_sample = np.clip(X_data[i].astype(np.float32) / 255.0, 0.0, 1.0)  # Normalize & Clip
            y_sample = np.clip(Y_data[i].astype(np.float32) / 255.0, 0.0, 1.0)
            yield x_sample, y_sample  

    return tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            tf.TensorSpec(shape=(None, None, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, None, 3), dtype=tf.float32)
        )
    )

batch_size = 8
# train_ds = data_generator("X_train.npy", "Y_train.npy").batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_ds = data_generator(X_train_path, Y_train_path).batch(batch_size).prefetch(tf.data.AUTOTUNE)
# Convert the Dataset objects to numpy arrays
val_x = []
val_y = []
# Iterate over the validation dataset and collect the data
for x_batch, y_batch in val_ds:
    val_x.append(x_batch.numpy())
    val_y.append(y_batch.numpy())

# Convert the lists to numpy arrays
val_x = np.concatenate(val_x, axis=0)
val_y = np.concatenate(val_y, axis=0)


In [4]:
def calculate_metrics(img1, img2):
    img1, img2 = img1.astype(np.float64), img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    mae = np.mean(np.abs(img1 - img2))
    psnr = 20 * np.log10(255.0 / np.sqrt(mse)) if mse != 0 else float('inf')
    ssim = np.mean((2 * img1 * img2 + 0.01 * 255) / (img1**2 + img2**2 + 0.01 * 255))
    return psnr, ssim

In [ ]:
from model.srgan import generator
from model import resolve_single
model = generator()v
model.load_weights('artifacts/srgan/gan_generator.h5')

In [ ]:
metrics = {"val_psnr": [],"val_ssim": []}
metric_prefix = 'val'
for i in range(len(val_x)):
    output = resolve_single(model, val_x[i])
    output = cv2.resize(output.numpy(), (val_x[i].shape[1], val_x[i].shape[0]), interpolation=cv2.INTER_CUBIC)
    # output = model.predict(val_x[i].reshape(1, *val_x[i].shape), verbose=0)[0]
    output = img_as_ubyte(np.clip(output, 0, 1))
    img_y = img_as_ubyte(val_y[i])
    psnr, ssim = calculate_metrics(img_y, output)
    metrics[f"{metric_prefix}_psnr"].append(psnr)
    metrics[f"{metric_prefix}_ssim"].append(ssim)

In [12]:
metrics_avg = {k: np.mean(v) for k, v in metrics.items()}  # Average the metrics
df_metrics = pd.DataFrame([metrics_avg])  # Create a DataFrame with a single row (average metrics)

In [13]:
df_metrics

,val_psnr,val_ssim
0,20.524074,0.928911
